In [6]:
import pandas as pd
import os, sys
import matplotlib.pyplot as plt


# TEST preprocessing:

In [8]:
ed = pd.read_csv('edu_test.csv') # education DataFrame
hh = pd.read_csv('hh_test.csv') # household DataFrame



In [9]:
### Prepare Ed data
def prepare_df(df, suffix:str):
    uids = df['psu'].astype(str) + "_"  + df['hh'].astype(str) + "_" + df['idcode'].astype(str) 
    df = df.drop(columns=['psu', 'hh', 'idcode'])
    df.columns = [suffix + "_" + col.capitalize() for col in df.columns]
    df.insert(0, 'uid', uids)
    return df

ed = prepare_df(ed, 'ED')
hh = prepare_df(hh, 'HH')

# Remove absoluteley useless variates

In [11]:
hh = hh.drop(columns=['HH_Hhid', 'HH_Q04', 'HH_Q08', 'HH_Q12', 'HH_Q18'])
columns_to_keep = [
    'uid',
    'ED_Q01', 'ED_Q02', 'ED_Q03', 'ED_Q04', 'ED_Q05', 'ED_Q06', 'ED_Q07', 'ED_Q08', 'ED_Q09', 'ED_Q10', 'ED_Q11', 
    'ED_Q14', 'ED_Q15', 'ED_Q16', 'ED_Q17', 'ED_Q18',
    'ED_Q19',
    'ED_Q23',
    'ED_Q26', 'ED_Q27', 'ED_Q28', 'ED_Q29',
    'ED_Q41', 
]

ed = ed[columns_to_keep]


# Rename variates for interpretability

In [13]:
# ED variates
ed['literate'] = ed['ED_Q01'].eq(1) & ed['ED_Q02'].eq(1) 

ed['attended_school'] = ed['ED_Q03'] 

ed['highest_school_lvl'] = ed['ED_Q04'] 

ed['preschool'] = ed['ED_Q07']

# Merge
ed['now_enrolled'] = ed['ED_Q08']
ed['now_attending'] = ed['ED_Q09'] # 1 yes, 2 No
ed['past_enrolled'] = ed['ED_Q14'] # 1 yes, 2 No
ed['past_attending'] = ed['ED_Q15'] # 1 yes, 2 No

# Merge
ed['now_not_attend_reason'] = ed['ED_Q10']
ed['now_not_enroll_reason'] = ed['ED_Q11']
ed['past_not_attend_reason'] = ed['ED_Q16'] 
ed['past_not_enroll_reason'] = ed['ED_Q17']

ed['now_not_attend_reason'] = pd.Categorical(ed['now_not_attend_reason'], categories=list(range(1, 15)))
ed['now_not_enroll_reason'] = pd.Categorical(ed['now_not_enroll_reason'], categories=list(range(1, 15)))
ed['past_not_attend_reason'] = pd.Categorical(ed['past_not_attend_reason'], categories=list(range(1, 15)))
ed['past_not_enroll_reason'] = pd.Categorical(ed['past_not_enroll_reason'], categories=list(range(1, 15)))


ed['finish_school_age'] = ed['ED_Q18'] 

# ed['younger_19'] = ed['ED_Q19']

ed['public_private_school'] = ed['ED_Q23'] 
ed['public_private_school'] = pd.Categorical(ed['public_private_school'], categories=list(range(1, 4)))


ed['school_transportation_time'] = ed['ED_Q26'].apply(lambda hrs : hrs * 60) +  ed['ED_Q27'] # it works

####
trans = [0, 1, 2,0, 3, 4]
ed['school_transportation_vehicle'] = ed['ED_Q28'].apply(lambda indx : indx if pd.isna(indx) else trans[int(indx)])
ed['school_transportation_vehicle'] = pd.Categorical(ed['school_transportation_vehicle'], categories=list(range(1, 5)))


####
ed['school_transportation_cost'] = ed['ED_Q29'] 

ed['school_expenses'] = ed['ED_Q41'] 


In [14]:
new_cols = [
    'uid',
    'literate','attended_school','highest_school_lvl',
    'preschool',
    'now_enrolled','now_attending','past_enrolled', 'past_attending','now_not_attend_reason',
    'now_not_enroll_reason','past_not_attend_reason','past_not_enroll_reason',
    'finish_school_age',
    'public_private_school',
    'school_transportation_time','school_transportation_vehicle',
    'school_transportation_cost','school_expenses'
]


ed = ed[new_cols]


In [15]:

# HH Variates. A random forest might atually be good
hh['sex'] = hh['HH_Q02'] 
hh['family_role'] = hh['HH_Q03'] 
hh['family_role'] = pd.Categorical(hh['family_role'], categories=list(range(1, 15)))

hh['age'] = hh['HH_Q05y'] # don't need month decimal

hh['marital_status'] = hh['HH_Q06'] 
hh['marital_status'] = pd.Categorical(hh['marital_status'], categories=list(range(1, 6)))


hh['spouse_lives'] = hh['HH_Q07']
hh['time_away'] = hh['HH_Q09'] 
# hh[''] = hh['HH_Q10'] 

# leave the sorting to the tree. that's what ti does better than anybody!!!! 
hh['lives_with_mom'] = hh['HH_Q11'] 
hh['moms_education'] = hh['HH_Q13'] 
hh['mom_alive'] = hh['HH_Q14'] 

hh['lives_with_dad'] = hh['HH_Q17'] 
hh['dad_education'] = hh['HH_Q19'] 
hh['dad_alive'] = hh['HH_Q20']

hh = hh.drop(columns=['HH_Q02', 'HH_Q03', 'HH_Q05y', 'HH_Q05m', 'HH_Q06', 'HH_Q07',
       'HH_Q09', 'HH_Q10', 'HH_Q11', 'HH_Q13', 'HH_Q14', 'HH_Q15', 'HH_Q16',
       'HH_Q17', 'HH_Q19', 'HH_Q20', 'HH_Q21', 'HH_Q22'])


# Preliminary Variable Selection!!!!


# Is student?

In [18]:
# enrollment and attendance past and present:

ed['now_attending'] = ed['now_attending'].fillna(ed['now_enrolled'])
ed['past_attending'] = ed['past_attending'].fillna(ed['past_enrolled'])

# We only care if they attended at least one year in the past 2.
attended = ed['now_attending'].eq(1) | ed['past_attending'].eq(1)  # NaN values mean they are NOT students
ed.loc[attended, 'is_student'] = True
ed.loc[~attended, 'is_student'] = False

ed =  ed.drop(columns=['now_enrolled', 'past_enrolled', 'past_attending', 'now_attending'])


# absence reasons:

In [20]:

ed.loc[ed['now_not_attend_reason'] == 13, 'now_not_attend_reason'] = 14
ed.loc[ed['past_not_attend_reason'] == 13, 'past_not_attend_reason'] = 14

# We have situations where someone enrolls but does not attend.
# We don't have situations when someone who did not enroll attends. 
# So use attend as base truth and fill some of the NaNs with enrolled.
ed['now_not_attend_reason'] = ed['now_not_attend_reason'].fillna(ed['now_not_enroll_reason'])
ed['past_not_attend_reason'] = ed['past_not_attend_reason'].fillna(ed['past_not_enroll_reason'])

ed = ed.drop(columns=['now_not_enroll_reason', 'past_not_enroll_reason'])

ed['now_not_attend_reason'] = ed['now_not_attend_reason'].fillna(14) #"other"
ed['past_not_attend_reason'] = ed['past_not_attend_reason'].fillna(14) # other

In [21]:
dummies = pd.get_dummies(ed['now_not_attend_reason'], prefix='not_attend_reason1', drop_first=True)
ed = pd.concat([ed, dummies], axis=1)

dummies = pd.get_dummies(ed['past_not_attend_reason'], prefix='not_attend_reason2', drop_first=True)
ed = pd.concat([ed, dummies], axis=1)



In [22]:
ed = ed.drop(columns=['now_not_attend_reason', 'past_not_attend_reason'])

In [23]:
# cols = ['now_not_attend_reason', 'past_not_attend_reason']
# data = ed[cols]
# data.loc[data['now_not_attend_reason'].ne(data['past_not_attend_reason']), cols]

In [24]:
# # Now apply transform reasons to the financial scale
# ABSENCE_REASON_TO_MONEY_MAPPING = [0, 3,1,3,3,1,1,1,2,2,1,1,2,1,1]

# ed['now_not_attend_reason'] =  ed['now_not_attend_reason'].apply(lambda indx : ABSENCE_REASON_TO_MONEY_MAPPING[int(indx)])
# ed['past_not_attend_reason'] =  ed['past_not_attend_reason'].apply(lambda indx : ABSENCE_REASON_TO_MONEY_MAPPING[int(indx)])

In [25]:
# Now the merge between now_not_attend_reason and past_not_attend_reason becomes obvious for our purposes of predicting wealth: pick the maximum. 
# ed['not_attend_reason'] =  ed.apply(lambda row: max(row['past_not_attend_reason'], row['now_not_attend_reason']), axis=1)


In [26]:
# ed = ed.drop(columns=['now_not_attend_reason', 'past_not_attend_reason'])

In [27]:
# # Now we need to merge Missed_year? and not_attend_reason into one variate

# ## CLAIM: Missed_year==False <-> not_attend_reason=0
# claim = ed['Missed_year'].eq(False) & ed['not_attend_reason'].ne(0)
# assert(len(ed[claim])==0)

# # DONE: So they are already dependent!! No need to do anything other than delete missed_year?

# # The meaning of this not_attend_reason variable in the FINAL TRANSFORM ED_T4 is: 
# # It represents what happened in the last couple of years
# # 0 if the kid attended both years to school
# # 1 if the kid missed at least one year due to something, in the worse case scenario, (NOT) related to money
# # 2 if the kid missed at least one year due to something, in the worse case scenario, (SOMEWHAT) related to money
# # 3 if the kid missed at least one year due to something, in the worse case scenario, (DIRECTLY) related to money


In [28]:
# ed = ed.drop(columns=['Missed_year'])

# Highest School lvl

In [30]:
Q4_education_mapping = [0, 1, 2, 3, 4, 4, 5, 5, 6, 6, 7, 7] # just gives the same values to same degrees (no matter where they were studied)
ed['highest_school_lvl'] =  ed['highest_school_lvl'].apply(lambda indx : indx if pd.isna(indx) else Q4_education_mapping[int(indx)])

## fill out NaN values with standarized mean
av = ed['highest_school_lvl'].mean()
ed['highest_school_lvl'] = ed['highest_school_lvl'].fillna(round(av, 2))


# Preschool?

In [32]:
cond = ed['preschool'].isna()
ed['preschool'] = ed['preschool'].fillna(0)

# finish_school_age, younger_19
 don't make sense. we will have the age from hh

In [34]:
av = round(ed['finish_school_age'].mean(), 2)
ed['finish_school_age'] = ed['finish_school_age'].fillna(av)

# public_private_school
Q23: Is the school that [Name] attend in public or private?

* PUBLIC	= 1
* PRIVATE RELIGIOUS	= 2
* PRIVATE NON-RELIGIOUS	= 3

I like this ordering because, apparently, "Non-religious private schools are more expensive than religious private schools on average"

NaN replacement: 
AVERAGE because average assigns a more conservative value to NaN. A zero on this new scale is still the 25th quantile so it is biased towards poverty ya know.

But indicator variables have the potential to model the relatinsip better than by increases in 1... 
**everytime you want to make up a scale think of this: can I do it bettern than a machine?**

In [36]:
ed['public_private_school'] = ed['public_private_school'].fillna(1) # no data avaliable
dummies = pd.get_dummies(ed['public_private_school'], prefix='school_type', drop_first=True)
ed = pd.concat([ed, dummies], axis=1)


In [37]:
ed = ed.drop(columns=['public_private_school'])

# Transportation !!!!

Just transform into indicator variables!!!!! see what happens 

In [39]:
dummies = pd.get_dummies(ed['school_transportation_vehicle'], prefix='transport', drop_first=True)
ed = pd.concat([ed, dummies], axis=1)


In [40]:
ed = ed.drop(columns=['school_transportation_vehicle'])

In [41]:
# transportation_mapping = [float('nan'), 2, 1, float('nan'), 4, 3, float('nan'), float('nan')]

# ed['school_transportation_vehicle'] = ed['school_transportation_vehicle'].apply(lambda indx: indx if pd.isna(indx) else transportation_mapping[int(indx)] )
# av = int(ed['school_transportation_vehicle'].mean())
# ed['school_transportation_vehicle'] = ed['school_transportation_vehicle'].fillna(av)



In [42]:
av = int(ed['school_transportation_time'].mean())
ed['school_transportation_time'] = ed['school_transportation_time'].fillna(round(av,2))

# because they are kids so they probably feel like the cost is 0
ed['school_transportation_cost'] = ed['school_transportation_cost'].fillna(0)

# school_expenses

In [44]:
av = int(ed['school_expenses'].mean())
ed['school_expenses'] = ed['school_expenses'].fillna(round(av,2))


#### DONE!!! with ed

# Household data

In [47]:
# leave sex as is
hh['sex'] = hh['sex'].eq(1)


# family role

In [49]:
dummies = pd.get_dummies(hh['family_role'], prefix='fam_role', drop_first=True)
ed = pd.concat([ed, dummies], axis=1)


# marital_status 
just make it into 5 indicator variables. 



In [51]:
dummies = pd.get_dummies(hh['marital_status'], prefix='marital_status', drop_first=True)
ed = pd.concat([ed, dummies], axis=1)


In [52]:
# get rid of both
hh = hh.drop(columns=['marital_status', 'family_role'])

In [53]:
hh['lives_with_partner'] = hh['spouse_lives'].eq(1)
hh = hh.drop(columns=['spouse_lives'])

# time away is good

In [55]:
hh['lives_with_parent'] = hh['lives_with_mom'].eq(1) | hh['lives_with_dad'].eq(1)
hh = hh.drop(columns=['lives_with_mom', 'lives_with_dad'])

In [56]:
hh['parents_education'] = (hh['moms_education'] + hh['dad_education']) / 2
ave = hh['parents_education'].mean()
hh['parents_education'] = hh['parents_education'].fillna(round(ave,2))

hh = hh.drop(columns=['moms_education', 'dad_education'])

In [57]:
# parents alive 
hh['mom_alive'] = hh['mom_alive'].eq(1)
hh['dad_alive'] = hh['dad_alive'].eq(1)
hh['mom_alive'] = hh['mom_alive'].apply(lambda boo: 1 if boo else 0)
hh['dad_alive'] = hh['dad_alive'].apply(lambda boo: 1 if boo else 0)

hh['num_parents_alive'] = hh['mom_alive'] + hh['dad_alive']
hh = hh.drop(columns=['mom_alive', 'dad_alive'])

# DONE

In [59]:
df = pd.merge(ed, hh, on='uid', how='inner')  # Use 'inner' join by default


In [60]:
df.to_csv('X_TEST.csv', index=False)
